# Overview #
This notebook pulls data for selected stock list **ticks** and determines some fundamentals to help evaluate each stock 
in the list.

In [2]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime as dt
import bs4 as bs
import requests

ek.set_app_id('DeNovoQuantFund')

### Stock List ###

In [ ]:
# List of S&P 500 tickers

def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    
    return tickers

ticker_list = save_sp500_tickers()

In [17]:
# ticker_list = pd.read_csv(filepath_or_buffer='/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_mid_2018_review/NASDAQ_top_choices_with_pe_sales.csv')
# ticker_list = ['CMCSA.O','MPWR.O','CBRL.O','SWKS.O','ULTI.O','AMWD.O','UTHR.O']
ticker_list = ['CM.TO', 'TD.TO', 'BMO.TO']
# ticker_list = ['LNR.TO','WJA.TO', 'FB.O','TD.TO', 'ALA.TO','L.TO',
#                'NTES.O','SNAP.N','TXN.O','AAPL.O','XST.TO','RHT',
#                'RDSa.AS','T.TO', 'SWKS.O','CM.TO', 'INTU.O', 'GOOG.O', 'SHOP.TO'
#               'CMCSA.O','MPWR.O','CBRL.O','SWKS.O','ULTI.O','AMWD.O','UTHR.O', 'ECA.TO']
#['COHR.O','FB.O','MAA','GOOG.O','NTES.O','CM.TO','TD.TO','TU']
               

### Getting Relavent Fields ###
These are the fundamental feilds I grab to assess if a company is undervalued.

In [3]:
f1 = [ek.TR_Field('TR.PE.date'), ek.TR_Field('TR.PriceToSalesPerShare'), ek.TR_Field('TR.PE'),
              ek.TR_Field('TR.PriceToBVPerShare'),ek.TR_Field('TR.DPSActValue'),ek.TR_Field('TR.DilutedEpsExclExtra'),
      ek.TR_Field('TR.PriceToBVPerShare'), ek.TR_Field('TR.BookValuePerShare'), ek.TR_Field('TR.RevenuePerShare'),
      ek.TR_Field('TR.TtlDebtToTtlEquityPct'), ek.TR_Field('TR.EBITMarginPercent'),
              ek.TR_Field('TR.ROATotalAssetsPercent'), ek.TR_Field('TR.TotalDebtToEV'),ek.TR_Field('TR.TotalDebtToEBITDA')
     ]

f2 = [ek.TR_Field('TR.OperatingIncome'),ek.TR_Field('TR.Revenue'),ek.TR_Field('TR.NormIncAvailToCommon'), 
      ek.TR_Field('TR.GrossDividendsCmnStock'), ek.TR_Field('TR.TotalCurrentAssets'),
              ek.TR_Field('TR.TotalCurrLiabilities'), ek.TR_Field('TR.TotalAssetsReported'), ek.TR_Field('TR.TotalLiabilities'),
              ek.TR_Field('TR.TotalLongTermDebt'), ek.TR_Field('TR.TotalEquity'), 
              ek.TR_Field('TR.EBIT')]


start_date = '2009-01-01'
end_date = dt.datetime.now()
end_date = end_date.strftime('%Y-%m-%d')

f = f1 + f2

## Getting Quarterly and Yearly Fundamental Data

In [4]:
#Getting Start and End Dates

start_date = dt.datetime(2009,1,1)
end_date = dt.datetime.today()
start_date1 = start_date.strftime('%Y-%m-%d')
end_date1 = end_date.strftime('%Y-%m-%d')

### Organize and Aggregate Pulled Data ###
This function pulls the data of interest and aggregates key fundamentals by year that ratio analysis can be done on.
(Sorry, it's not the cleanest code, but it does the job.)

In [18]:
def get_organized_data(ticker, start_date = '2009-01-01', end_date ='2018-01-01'):
    ticker = str(ticker)
    ticker1 = str(ticker)
    try:
        df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':start_date, 'EDate': end_date, 
                                                              'Frq': 'D'})
        
        print("Error Reached")
        print(type(df_fundamental))
        # Aggregate data
        df_fundamental.sort_values(axis=0, by='Date', inplace= True)
        df_fundamental = df_fundamental[df_fundamental['Date'] !='']
        df_fundamental['Date'] = pd.to_datetime(df_fundamental['Date'])
        # Group data by the mean value of each year.
        df_grouped = df_fundamental.groupby(by=df_fundamental['Date'].dt.year).mean()
        
        # cur_prices = ek.get_timeseries([ticker],fields= 'CLOSE',
                   #   end_date= dt.datetime.today().strftime('%Y-%m-%d'))
        
        return [df_grouped, df_fundamental, True]
    
    except:
        try:
            ticker = ticker1 + '.O'
            df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':start_date, 'EDate': end_date, 
                                                                  'Frq': 'D'})
            
            # Aggregate data
            df_fundamental.sort_values(axis=0, by='Date', inplace= True)
            df_fundamental = df_fundamental[df_fundamental['Date'] !='']
            df_fundamental['Date'] = pd.to_datetime(df_fundamental['Date'])
            df_grouped = df_fundamental.groupby(by=df_fundamental['Date'].dt.year).mean()
            
            #cur_prices = ek.get_timeseries([ticker],fields= 'CLOSE',
                      #    end_date= dt.datetime.today().strftime('%Y-%m-%d'))
            return [df_grouped, df_fundamental, True]
            
        except:
            try:
                ticker = ticker1 + '.OQ'
                df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':start_date, 'EDate': end_date, 
                                                                      'Frq': 'D'})
                
                # Aggregate data
                df_fundamental.sort_values(axis=0, by='Date', inplace= True)
                df_fundamental = df_fundamental[df_fundamental['Date'] !='']
                df_fundamental['Date'] = pd.to_datetime(df_fundamental['Date'])
                df_grouped = df_fundamental.groupby(by=df_fundamental['Date'].dt.year).mean()
                
                #cur_prices = ek.get_timeseries([ticker],fields= 'CLOSE',
                    #  end_date= dt.datetime.today().strftime('%Y-%m-%d'))
                return [df_grouped, df_fundamental, True]
            
            except:
                try:
                    ticker = ticker1 + '.N'
                    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate': start_date, 'EDate': end_date, 
                                                                          'Frq': 'D'})
                    
                    # Aggregate data
                    df_fundamental.sort_values(axis=0, by='Date', inplace= True)
                    df_fundamental = df_fundamental[df_fundamental['Date'] !='']
                    df_fundamental['Date'] = pd.to_datetime(df_fundamental['Date'])
                    df_grouped = df_fundamental.groupby(by=df_fundamental['Date'].dt.year).mean()
                    #cur_prices = ek.get_timeseries([ticker],fields= 'CLOSE',
                        #  end_date= dt.datetime.today().strftime('%Y-%m-%d'))
                    return [df_grouped, df_fundamental, True]
                
                except:
                    print('Ticker {} not found'.format(ticker))
                    return [0,0,False]


In [27]:
# Quickly testing out data is being pulled correctly.
df_fundamental, err = ek.get_data(['AAPL.O'], fields= f,parameters={'SDate':'2009-01-01', 'EDate': '2018-01-01', 
                                                                  'Frq': 'D'})

In [30]:
print(err is None)

True


## Benjamin Graham Metrics ##
Benjamin Graham is the father of value investing (he was Warren Buffet's teacher). To say he has done a lot of great work in this space would be an understandment.

Graham came up with 8 metrics that you can use to easily assess the value of a company. These metrics are outlined below. Please read the comments!

In [21]:
def get_BG_metrics(df_grouped,df_fundamental):
    
    cur_year = df_grouped.index.max()
    
    #1. Sales above 500 million
    sales = df_grouped.loc[cur_year,'Revenue'] > 500e6
    sales
    #2. Current assets twice current liabilities
    cur_ass_to_cur_liab = df_grouped.loc[cur_year, 'Total Current Assets']/df_grouped.loc[cur_year, 'Total Current Liabilities'] >= 2
    cur_ass_to_cur_liab
    
    #3. working capital > long term debt
    long_term_debt_vs_net_current_assets = (df_grouped.loc[cur_year,'Total Current Assets']-
                                            df_grouped.loc[cur_year,'Total Current Liabilities']> 
                                            df_grouped.loc[cur_year,'Total Long Term Debt'])
    long_term_debt_vs_net_current_assets
    
    #4. Positive Earnings for past 10 years
    all_pos_earnings = all(df_grouped.loc[:,'Normalized Income Avail to Cmn Shareholders']> 0)
    all_pos_earnings
    
    #5. Div Payments for past 20 year (I will use 8 years)
    div_pos_earnings = all(df_grouped.loc[:,'Dividend Per Share - Actual']> 0)
    div_pos_earnings
    
    #6. Earnings increased by 1/3rd in past 10 years
    sufficient_earnings_increase = (df_grouped.loc[cur_year,'Normalized Income Avail to Cmn Shareholders'] - 
                                   df_grouped.loc[df_grouped.index.min(),'Normalized Income Avail to Cmn Shareholders'])/abs(df_grouped.loc[df_grouped.index.min(),'Normalized Income Avail to Cmn Shareholders'])>= 1.33 
    sufficient_earnings_increase
    
    #7. Current Price <= 15*EPS  
    cur_price_less_15_eps = df_grouped.loc[cur_year,'P/E (Daily Time Series Ratio)'] <= 15
    cur_price_less_15_eps
    
    #8. Current value <= 1.5 Book value
    cv_less_bv = df_grouped.loc[cur_year,'Price To Book Value Per Share (Daily Time Series Ratio)'][0] <= 1.5
    cv_less_bv
    
    bg_criteria = pd.DataFrame([sales,cur_ass_to_cur_liab,long_term_debt_vs_net_current_assets,
                               all_pos_earnings,div_pos_earnings,sufficient_earnings_increase,
                               cur_price_less_15_eps,cv_less_bv], 
                              index = ['Sales above 500M','Cur Assets >= 2*Cur Liabilities',
                                      'Working Cap >= Long term debt','Pos Earnings for past 8 years',
                                      'Pos Div Earnings past 8 years','Increase in earnings by 33% over past 8 years',
                                      'P/E <= 15', 'Price to Book Val <= 1.5'])
    return bg_criteria

## Kahn Metrics ##
I decided to add a set of metrics myself based on my experience and knowledge that I thought would complement Graham's and add some additional criteria that I wanted in all the companies I invest in. The metrics are explained in the comments.

In [22]:
def get_ZK_metrics(df_grouped, df_fundamental):
    kahn_params1 = df_grouped.drop(labels= ['Price To Sales Per Share (Daily Time Series Ratio) ',
                                   'P/E (Daily Time Series Ratio)',
                                   'Price To Book Value Per Share (Daily Time Series Ratio)',
                                   'Dividend Per Share - Actual'], axis = 1)
    # Get percent change between in year.
    delta_data = kahn_params1.pct_change()*100
    
    #pe_relatively_low = df_fundamental.iloc[-1,3] < df_grouped['P/E (Daily Time Series Ratio)'].mean()
    
    if df_fundamental.iloc[-1:,3].isnull().all(): # Checks if the current p/e value is nan 
        if df_fundamental.iloc[-100:,3].isnull().all(): # Checks if all the p/e values for the past 100 days are null
            # If yes, then grab p/e average for the year
            # Look for P/E ratios that are currently less than the historical average.
            # Why? Because this shows are willing to pay less for every dollar of earnings right now than they
            # historically have. This could represent a buying opportunity.
            pe_relatively_low = df_grouped.loc[df_grouped.index.max(),'P/E (Daily Time Series Ratio)']< df_grouped['P/E (Daily Time Series Ratio)'].mean()
            pe_sale =df_grouped.loc[df_grouped.index.max(),'P/E (Daily Time Series Ratio)']/df_grouped['P/E (Daily Time Series Ratio)'].mean()
            
        else: # Grab mean p/e for past 100 days.

            pe_relatively_low = df_fundamental.iloc[-100:,3].mean()< df_grouped['P/E (Daily Time Series Ratio)'].mean()
            pe_sale =df_fundamental.iloc[-100:,3].mean()/df_grouped['P/E (Daily Time Series Ratio)'].mean()
            
    
    else:
    
        pe_relatively_low = df_fundamental.iloc[-1,3] < df_grouped['P/E (Daily Time Series Ratio)'].mean()
        pe_sale = df_fundamental.iloc[-1,3]/df_grouped['P/E (Daily Time Series Ratio)'].mean()
    
    
    # Check EBITDA margin never decreases by more than 2.5% year over year.
    delta_ebitda_margin = all(   delta_data.loc[:,'EBIT Margin, Percent']> -2.5)
    #delta_roa_margin = all(   kahn_params1.loc[:,'ROA Total Assets, Percent']> -2.5)
    # Check ROA margin always increases by more than 1.5%  year over year.
    delta_roa_margin = kahn_params1.loc[:,'ROA Total Assets, Percent'].mean()> 1.5
    # Check current ratio (current assets/current liabilities) > 1.5
    current_ratio = float(df_fundamental.iloc[-1,19])/df_fundamental.iloc[-1,20] >= 1.5
    # Check earnings have increased on average by over 6.5% for the past 7 years.
    earnings_increase_for_past_7_years = delta_data.iloc[:-1,11].dropna().mean() >= 6.5
    # Check revunue has increased on average by over 6.5% for the past 7 years.
    rev_increase_for_past_7_years = delta_data.iloc[:-1,10].dropna().mean() >= 6.5
    # Check share holders equity has increased on average by over 6.5% for the past 7 years.
    equity_increase_for_past_7_years = delta_data.iloc[:-1,18].dropna().mean() >= 6.5
    
    # Check debt to equity is increasing no more than 3% a year.
    if (delta_data.iloc[:,4].isin([-100]).any() == True) or (kahn_params1.iloc[:,4].isin([0]).any() == True):
        
        decreasing_debt_to_equity = True
    else:
        decreasing_debt_to_equity = delta_data.iloc[:,4].dropna()
        decreasing_debt_to_equity = decreasing_debt_to_equity[decreasing_debt_to_equity.isin([np.inf]) == False]
        decreasing_debt_to_equity = decreasing_debt_to_equity.mean() <= 3
    
    # Check debt to EBIT is increasing by no more than 10% a year.
    if delta_data.iloc[:,8].isin([-100]).any() == True:
        
        decreasing_debt_to_ebit = True
    else:
        decreasing_debt_to_ebit = delta_data.iloc[:,8].dropna()
        decreasing_debt_to_ebit = decreasing_debt_to_ebit[decreasing_debt_to_ebit.isin([np.inf]) == False]
        decreasing_debt_to_ebit = decreasing_debt_to_ebit.mean() <= 10
        
        
        
    kahn_value_metrics = pd.DataFrame(data= [pe_relatively_low,delta_ebitda_margin,delta_roa_margin,
                                            current_ratio,earnings_increase_for_past_7_years,
                                            rev_increase_for_past_7_years,equity_increase_for_past_7_years,
                                            decreasing_debt_to_equity,decreasing_debt_to_ebit],
                                     index = ['Relatively low PE','Ebitda Margin','ROA Margin',
                                             'Current Ratio','Earnings Increase',
                                             'Revenue Increase','Equity Increase','Decreasing Debt to Equity',
                                             'Decreasing Debt to Ebit'])
    
    #pe_sale = df_fundamental.iloc[-1,3]/df_grouped['P/E (Daily Time Series Ratio)'].mean()
    #print('avg P/E')
    #print(df_grouped['P/E (Daily Time Series Ratio)'].mean())
    
    return [kahn_value_metrics, pe_sale]

## Going through stock list ##

In [23]:
stock_list_low_pe =[]
pe_sales = []
score = []
stock_list = []

final_overview = {}

for i,tic in enumerate(ticker_list):
    try:
        [df_grouped, df_fundamental,data_found] = get_organized_data(tic,start_date=start_date1,end_date=end_date1)
        
        
        print data_found
        
        if data_found == True:
            bg_criteria = get_BG_metrics(df_grouped,df_fundamental)
            [kahn_value_metrics, pe_sale] = get_ZK_metrics(df_grouped, df_fundamental)
            
            
            
            final_overview[tic] = [kahn_value_metrics,pe_sale,bg_criteria]
            # Final Decision
    
            final_decision = bg_criteria.sum().iloc[0] + kahn_value_metrics.sum().iloc[0] >= 9 and kahn_value_metrics.loc['Relatively low PE'] == True
            
            
            
            if isinstance(final_decision, pd.Series):
                final_decision = final_decision.bool()
            
            
            
            if final_decision == True:
                stock_list_low_pe.append(tic)
                pe_sales.append(pe_sale)
                score.append(bg_criteria.sum().iloc[0] + kahn_value_metrics.sum().iloc[0])
                print(stock_list_low_pe)
                
            
                
            final_decision2 = bg_criteria.sum().iloc[0] + kahn_value_metrics.sum().iloc[0] >= 10
            
            if isinstance(final_decision2, pd.Series):
                final_decision2 = final_decision2.bool()
                
            
            
            if final_decision2 == True:
                print 'made it'
                stock_list.append(tic)
                print(stock_list)
                
            
    except:
        print('An error occurred for {}'.format(tic))
        
    print(i)

True
0
True
['TD.TO']
1
True
['TD.TO', 'BMO.TO']
made it
['BMO.TO']
2


## Reviewing Stocks ##

In [9]:
made_the_cut = pd.DataFrame(stock_list)

In [ ]:
made_the_cut.to_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_mid_2018_review/NASDAQ_made_the_cut.csv')

In [10]:
under_priced = pd.DataFrame({'tickers': stock_list_low_pe, 'pe_sale': pe_sales})

In [ ]:
under_priced.to_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_mid_2018_review/NASDAQ_made_the_cut_low_pe.csv')

In [11]:
under_priced

,pe_sale,tickers
0,0.453115,LNR.TO
1,0.209520,FB.O
2,0.900528,TD.TO
3,0.409697,ALA.TO
4,0.712960,L.TO
5,0.985236,TXN.O
6,0.601806,SWKS.O
7,0.968556,GOOG.O
8,0.601806,SWKS.O
9,0.462374,ULTI.O


## Finding Companies with strong fundamentals and PE sale

In [ ]:
made_the_cut = pd.read_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_Ticker_Symbols_made_cut.csv')
under_priced = pd.read_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_Ticker_Symbols_low_pe.csv')

In [ ]:
made_the_cut.rename(columns={'0':'tickers'},inplace=True)

In [13]:
under_priced

,pe_sale,tickers
0,0.460043,LNR.TO
1,0.158115,FB.O
2,0.853553,TD.TO
3,0.860931,L.TO
4,0.813364,AAPL.O
5,0.531031,SWKS.O
6,0.948988,GOOG.O
7,0.753559,MPWR.O
8,0.531031,SWKS.O
9,0.350063,ULTI.O


In [24]:
final_overview

{'BMO.TO': [                               0
  Relatively low PE           True
  Ebitda Margin              False
  ROA Margin                 False
  Current Ratio              False
  Earnings Increase           True
  Revenue Increase           False
  Equity Increase             True
  Decreasing Debt to Equity   True
  Decreasing Debt to Ebit     True,
  0.7969070841653855,
                                                     0
  Sales above 500M                               False
  Cur Assets >= 2*Cur Liabilities                False
  Working Cap >= Long term debt                  False
  Pos Earnings for past 8 years                   True
  Pos Div Earnings past 8 years                   True
  Increase in earnings by 33% over past 8 years   True
  P/E <= 15                                       True
  Price to Book Val <= 1.5                        True],
 'CM.TO': [                               0
  Relatively low PE           True
  Ebitda Margin              False
  ROA 

### Figuring out stuff

In [19]:
[df_grouped, df_fundamental,data_found] = get_organized_data('SHOP.TO',end_date=end_date1)

Ticker SHOP.TO.N not found


In [21]:
df_grouped

,Price To Sales Per Share (Daily Time Series Ratio),P/E (Daily Time Series Ratio),Price To Book Value Per Share (Daily Time Series Ratio),Dividend Per Share - Actual,Diluted EPS Excluding Extraordinary Items,Price To Book Value Per Share (Daily Time Series Ratio),Book Value Per Share,Revenue Per Share,"Total Debt to Total Equity, Percent","EBIT Margin, Percent",...,Revenue,Normalized Income Avail to Cmn Shareholders,Gross Dividends - Common Stock,Total Current Assets,Total Current Liabilities,"Total Assets, Reported",Total Liabilities,Total Long Term Debt,Total Equity,EBIT
Date,,,,,,,,,,,,,,,,,,,,,
2001,1.252728,11.397598,1.630187,NaN,4.651122,1.630187,29.437567,29.686382,168.521932,26.476340,...,NaN,1901056581,506808764,NaN,NaN,269177306772,257764968127,4380948207,11412338645,3153964143
2002,1.341261,15.810749,1.386833,1.600000,3.862457,1.386833,32.176636,29.625403,203.358212,37.472913,...,NaN,1688814904,539904761,NaN,NaN,286123428571,274182190476,3963571428,11941238095,4180333333
2003,1.677136,24.539010,1.458468,1.603810,1.704802,1.458468,34.608699,30.520178,112.912212,22.234427,...,NaN,919740873,578388888,NaN,NaN,273675341269,261198988095,3584341269,12476353174,2469785714
2004,2.298612,12.609400,1.742215,1.684269,4.953086,1.742215,37.582494,31.824134,164.343984,35.908690,...,NaN,1792264822,623292490,NaN,NaN,277421826086,263915422924,3314612648,13506403162,4136185770
2005,2.337741,13.158448,2.251621,2.406396,2.625512,2.251621,33.062280,34.405922,174.856482,31.141639,...,NaN,949233890,835579952,NaN,NaN,279488424821,267962031026,4436152744,11526393794,3742601431
2006,2.364529,12.417494,2.515685,2.678072,0.922622,2.515685,32.620583,35.887975,202.501570,28.995076,...,NaN,309939759,906240963,NaN,NaN,284922096385,273884397590,5197036144,11037698795,3506072289
2007,2.262101,11.798392,2.455988,2.782311,7.465085,2.455988,36.995355,33.683425,289.892050,53.582803,...,NaN,2526482071,932127490,NaN,NaN,306570844621,294169804780,5590326693,12401039840,6110374501
2008,1.457200,18.795422,1.581837,3.234731,4.206458,1.581837,39.214460,27.098604,277.916765,19.669216,...,NaN,1367016997,1123878186,NaN,NaN,346073138810,332470784702,5901195467,13602354107,3981005665
2009,2.281617,37.665447,1.753888,3.480000,-4.747990,1.753888,37.363663,12.155344,319.775564,-46.784315,...,NaN,-1753533333,1290733333,NaN,NaN,351531866666,337641666666,6457866666,13890200000,-1517000000


In [ ]:
get_ZK_metrics(df_grouped,df_fundamental)

In [ ]:
df_fundamental.iloc[-300:]